ФБ-24 Тихонова Анастасія Лабораторна робота 2

1. Створити env в якому будуть встановлені всі необхідні бібліотеки та налаштування для данної лабораторної роботи

conda create --name lab2 python
conda activate lab2
conda deactivate

2. Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження;

In [3]:
import urllib.request
import os
import datetime

province_ids = range(1, 28)  

base_url = 'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={}&year1=1981&year2=2024&type=Mean'

if not os.path.exists('csv_data'):
    os.makedirs('csv_data')

for province_id in province_ids:
    url = base_url.format(province_id)
    current_datetime = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    filename = 'VHI_{}_{}.csv'.format(province_id, current_datetime)
    filepath = os.path.join('csv_data', filename)

    try:
        urllib.request.urlretrieve(url, filepath)
    except Exception as e:
        print('Помилка при скачуванні файлу для provinceID {}: {}'.format(province_id, str(e)))

print("Усі файли скачано")

Усі файли скачано


3. Зчитати завантажені текстові файли у фрейм (детальніше про роботу із фреймами буде розказано у подальших лабораторних роботах). Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [29]:
import pandas as pd
import os

def clean(dirpath):
    ls = []

    if not os.path.exists(dirpath) or not os.path.isdir(dirpath):
        print("Директорії {dirpath} не існує, або вона не є директорією")
    else:
        files = os.listdir(dirpath)

        for i, file in enumerate(files):

            if file.endswith(".csv"):
                filepath = os.path.join(dirpath, file)
                df = pd.read_csv(filepath, index_col=False, header=1)
                df["ID"] = i + 1
                ls.append(df)
        df = pd.concat(ls).drop_duplicates().reset_index(drop=True)

    #print(list(df.columns.values))
    df = df.rename(columns={" VHI<br>": "VHI"})
    df = df.rename(columns={" SMN": "SMN"})

    #print(df.head())
    df.replace({"year": {"<tt><pre>1982": "1982"}}, inplace=True)
    df = df.drop(df.loc[df["VHI"] == -1].index)

    #print(df.tail())
    df = df.drop(60398)

    #print(df.dtypes)
    df["year"] = pd.to_numeric(df["year"], errors="coerce")
    

    #print(f'{df.head()}\n{df.dtypes}\n{list(df.columns.values)}\n{df.tail()}')

    selected_rec = df[(df["year"] == 2000) & (df["week"] == 18)]
    print("\nОбрандані за умовою: \n", selected_rec)

    #print(df)

clean("csv_data")


Обрандані за умовою: 
          year  week    SMN     SMT    VCI    TCI    VHI  ID
953    2000.0  18.0  0.382  291.46  87.96  40.29  64.13   1
3190   2000.0  18.0  0.321  291.75  62.07  44.97  53.52   2
5427   2000.0  18.0  0.258  291.37  57.87  35.04  46.45   3
7664   2000.0  18.0  0.274  294.94  58.05  43.47  50.76   4
9901   2000.0  18.0  0.230  293.42  48.85  43.10  45.98   5
12138  2000.0  18.0  0.422  289.89  89.19  46.43  67.81   6
14375  2000.0  18.0  0.292  296.09  58.30  49.03  53.67   7
16612  2000.0  18.0  0.306  296.63  58.66  40.70  49.68   8
18849  2000.0  18.0  0.280  292.93  56.17  39.62  47.89   9
21086  2000.0  18.0  0.372  290.84  82.90  38.86  60.88  10
23323  2000.0  18.0  0.309  292.81  58.99  45.14  52.06  11
25560  2000.0  18.0  0.376  290.30  80.00  60.17  70.09  12
27797  2000.0  18.0  0.341  291.00  77.63  41.08  59.36  13
30034  2000.0  18.0  0.368  292.21  81.01  35.14  58.07  14
32271  2000.0  18.0  0.399  288.67  70.34  39.18  54.76  15
34508  2000.0  1

4. Реалізувати процедуру, яка змінить індекси областей, які використані на порталі NOAA на наступні (виключно старі індекси на нові)

In [43]:
import pandas as pd
import os

def clean(dirpath):
    ls = []

    if not os.path.exists(dirpath) or not os.path.isdir(dirpath):
        print("Директорії {dirpath} не існує, або вона не є директорією")
    else:
        files = os.listdir(dirpath)

        for i, file in enumerate(files):

            if file.endswith(".csv"):
                filepath = os.path.join(dirpath, file)
                df = pd.read_csv(filepath, index_col=False, header=1)
                df["ID"] = i + 1
                ls.append(df)
        df = pd.concat(ls).drop_duplicates().reset_index(drop=True)

    #print(list(df.columns.values))
    df = df.rename(columns={" VHI<br>": "VHI"})
    df = df.rename(columns={" SMN": "SMN"})

    #print(df.head())
    df.replace({"year": {"<tt><pre>1982": "1982"}}, inplace=True)
    df = df.drop(df.loc[df["VHI"] == -1].index)

    #print(df.tail())
    df = df.drop(60398)

    #print(df.dtypes)
    df["year"] = pd.to_numeric(df["year"], errors="coerce")

    #print(df)
    return df

def index_replacement(df):
    new_indexes = {
        1: 22,
        2: 24,
        3: 23,
        4: 25,
        5: 3,
        6: 4,
        7: 8,
        8: 19,
        9: 20,
        10: 21,
        11: 9,
        12: 9,
        13: 10,
        14: 11,
        15: 12,
        16: 13,
        17: 14,
        18: 15,
        19: 16,
        20: 25,
        21: 17,
        22: 18,
        23: 6,
        24: 1,
        25: 2,
        26: 7,
        27: 5,
    }

    df["ID"] = df["ID"].replace(new_indexes)
    print(df)

index_replacement(clean("csv_data"))

         year  week    SMN     SMT    VCI    TCI    VHI  ID
0      1982.0   1.0  0.059  258.24  51.11  48.78  49.95  22
1      1982.0   2.0  0.063  261.53  55.89  38.20  47.04  22
2      1982.0   3.0  0.063  263.45  57.30  32.69  44.99  22
3      1982.0   4.0  0.061  265.10  53.96  28.62  41.29  22
4      1982.0   5.0  0.058  266.42  46.87  28.57  37.72  22
...       ...   ...    ...     ...    ...    ...    ...  ..
60357  2024.0  12.0  0.203  289.98  66.93   2.31  34.63   5
60358  2024.0  13.0  0.222  291.79  68.12   3.34  35.73   5
60359  2024.0  14.0  0.239  293.47  68.48   5.18  36.83   5
60360  2024.0  15.0  0.254  294.83  67.65   8.20  37.92   5
60361  2024.0  16.0  0.265  295.55  63.74  16.72  40.23   5

[58076 rows x 8 columns]


5. Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):

o Ряд VHI для області за вказаний рік;

o Пошук екстремумів (min та max) для вказаних областей та років;

o Ряд VHI за вказаний діапазон років для вказаних областей;

o Для всього набору даних виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25). 

o Аналогічно для помірних посух

In [75]:
import pandas as pd
import os

def clean(dirpath):
    ls = []

    if not os.path.exists(dirpath) or not os.path.isdir(dirpath):
        print("Директорії {dirpath} не існує, або вона не є директорією")
    else:
        files = os.listdir(dirpath)

        for i, file in enumerate(files):

            if file.endswith(".csv"):
                filepath = os.path.join(dirpath, file)
                df = pd.read_csv(filepath, index_col=False, header=1)
                df["ID"] = i + 1
                ls.append(df)
        df = pd.concat(ls).drop_duplicates().reset_index(drop=True)

    #print(list(df.columns.values))
    df = df.rename(columns={" VHI<br>": "VHI"})
    df = df.rename(columns={" SMN": "SMN"})

    #print(df.head())
    df.replace({"year": {"<tt><pre>1982": "1982"}}, inplace=True)
    df = df.drop(df.loc[df["VHI"] == -1].index)

    #print(df.tail())
    df = df.drop(60398)

    #print(df.dtypes)
    df["year"] = pd.to_numeric(df["year"], errors="coerce")

    #print(df)
    return df

def index_replacement(df):
    new_indexes = {
        1: 22,
        2: 24,
        3: 23,
        4: 25,
        5: 3,
        6: 4,
        7: 8,
        8: 19,
        9: 20,
        10: 21,
        11: 9,
        12: 9,
        13: 10,
        14: 11,
        15: 12,
        16: 13,
        17: 14,
        18: 15,
        19: 16,
        20: 25,
        21: 17,
        22: 18,
        23: 6,
        24: 1,
        25: 2,
        26: 7,
        27: 5,
    }

    df["ID"] = df["ID"].replace(new_indexes)
    return df

#Ряд VHI для області за вказаний рік
def VHIregionForYear(df, year, ProvinceID):
    vhiobl = df[(df.year == year) & (df["ID"] == ProvinceID)]["VHI"] 
    print(f"Ряд VHI для області {ProvinceID} у {year} році: ")
    print(vhiobl.tolist())


#Пошук екстремумів (min та max) для вказаних областей та років
def find_extremes(df, year, Province_ID):
    min_vhi = df[(df.year == year) & (df["ID"] == Province_ID)]["VHI"].min()
    max_vhi = df[(df.year == year) & (df["ID"] == Province_ID)]["VHI"].max()

    print(f"Мінімальне значення для області: {min_vhi}")
    print(f"Максимальне значення для області: {max_vhi}")


#Ряд VHI за вказаний діапазон років для вказаних областей
def VHIForRangeOfYears(df, start_year, end_year, Province_IDs):
        for year in range(start_year, end_year+1):
            for Province_ID in Province_IDs:
                vhi = df[(df.year == year) & (df["ID"] == Province_ID)]["VHI"]
                print(f"Ряд VHI для області {Province_ID} у {year} році:")
                print(vhi.tolist())


#Для всього набору даних виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка 
#областей по Україні (20% областей - 5 областей з 25).
def ExtremeDroughtYears(df, threshold_percentage):
    unique_years = df["year"].unique()
    e_drought_years = []

    for year in unique_years:
        extreme_drought_count = 0
        for Province_ID in range(1, 26):
            min_vhi = df[(df.year == year) & (df["ID"] == Province_ID)]["VHI"].min()
            if min_vhi <= 10:  
                extreme_drought_count += 1
        percentage = (extreme_drought_count / 25) * 100
        if percentage >= threshold_percentage:
            e_drought_years.append(year)

    print(f"Екстремальні посухи торкнулися більше {threshold_percentage}% областей України у наступні роки:")
    print(e_drought_years)



#Аналогічно для помірних посух
def ModerateDroughtYears(df, threshold_percentage):
    unique_years = df["year"].unique()
    m_drought_years = []

    for year in unique_years:
        moderate_drought_count = 0
        for Province_ID in range(1, 26):
            min_vhi = df[(df.year == year) & (df["ID"] == Province_ID)]["VHI"].min()
            if min_vhi <= 30 and min_vhi >= 20:
                moderate_drought_count += 1
        percentage = (moderate_drought_count / 25) * 100
        if percentage >= threshold_percentage:
            m_drought_years.append(year)

    print(f"Помірні посухи торкнулися більше {threshold_percentage}% областей України у наступні роки:")
    print(m_drought_years)


df = index_replacement(clean("csv_data"))

VHIregionForYear(df, 2012, 4)
print("----------------------------------")
find_extremes(df, 2015, 1)
print("----------------------------------")
VHIForRangeOfYears(df, 2000, 2003, [3, 5])
print("----------------------------------")
ExtremeDroughtYears(df, 20)
print("----------------------------------")
ModerateDroughtYears(df, 20)

Ряд VHI для області 4 у 2012 році: 
[43.16, 46.01, 49.89, 51.53, 52.67, 51.85, 50.22, 49.38, 49.07, 47.89, 46.03, 46.65, 47.25, 47.0, 47.94, 51.47, 54.3, 57.36, 59.62, 62.9, 65.95, 65.93, 62.43, 59.11, 55.86, 54.09, 52.88, 53.65, 54.06, 54.82, 54.02, 52.61, 51.98, 52.85, 53.57, 50.81, 49.04, 47.68, 46.75, 46.36, 46.44, 47.58, 51.61, 52.39, 50.52, 48.31, 47.06, 47.34, 45.3, 41.64, 42.07, 43.02]
----------------------------------
Мінімальне значення для області: 26.59
Максимальне значення для області: 57.58
----------------------------------
Ряд VHI для області 3 у 2000 році:
[38.66, 42.85, 45.94, 45.85, 43.7, 41.37, 38.91, 36.58, 34.2, 33.94, 33.84, 34.28, 33.53, 31.41, 30.92, 34.7, 40.25, 45.98, 50.52, 51.8, 50.31, 46.67, 48.27, 56.72, 65.44, 71.44, 72.12, 72.42, 73.2, 73.46, 74.06, 73.65, 70.76, 67.7, 66.77, 64.83, 62.63, 59.13, 55.18, 47.26, 40.04, 34.76, 31.95, 30.6, 27.93, 27.68, 26.34, 23.84, 22.92, 23.87, 26.29, 31.72]
Ряд VHI для області 5 у 2000 році:
[34.96, 34.07, 33.31, 32.9